In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Regresión lineal

Consiste en **ajustar** un modelo paramétrico
$$
f_\theta: x \rightarrow y
$$
que sea capaz de predecir $y$ dado $x$

- $x$ variable indepediente, entrada, característica, predictor
- $y$ variable dependiente, salida, respuesta, objetivo (target)
- $x$ e $y$ son variables continuas
- $\theta$ son los parámetros del modelo

> Encontrar como dos o más variables se relacionan. Explicar una variable en función de otras. Predicción

Hablamos de **regresión lineal** cuando el modelo $f_\theta$ es **lineal en sus parámetros**

¿Son estos modelos lineales en sus parámetros?
$$
\begin{align}
y &= f_\theta(x) = \theta_0  + \theta_1 x  \nonumber \\
y &= f_\theta(x) = \theta_0  + \theta_1 x + \theta_2 x^2 + \theta_3 \log(x) \nonumber \\
y &= f_\theta(x) = \theta_0  + \sin(\theta_1) x  \nonumber 
\end{align}
$$


- **Datos:** conjunto de $M$ tuplas $(\vec x_i, y_i)$ con $i=1,2,\ldots,M$ 
- **Ajuste:** Encontrar el valor óptimo de $\theta$ en función de los datos
- Podemos escribir el problema como un **sistema lineal de $M$ ecuaciones**
- Si el sistema es rectangular lo podemos resolver con **Mínimos Cuadrados**
- En dicho caso la solución es óptima *"en el sentido de mínimos cuadrádos"*

### Modelos lineales en sus parámetros y en sus entradas
#### Recta
Si $x$ es unidimensional 
$$
y =\theta_0  + \theta_1 x 
$$

#### Plano
Si $\vec x =(x_1, x_2)$ es bidimensional 
$$
y = \theta_0  + \theta_1 x_1 +  \theta_2 x_2 
$$

#### Hiperplano
Si $\vec x = (x_1, x_2, \ldots, x_d)$ es d-dimensional
$$
y = \theta_0  + \sum_{k=1}^d \theta_k x_k 
$$

In [ ]:
from ipywidgets import interact, SelectionSlider, IntSlider
from mpl_toolkits.mplot3d import Axes3D
plt.close('all'); fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection='3d')
theta = [4, 3, 2]; 

def update(rseed, N, sigma):
    ax.cla();
    np.random.seed(rseed);
    x1, x2 = np.random.randn(2, N)
    y_clean = theta[0] + theta[1]*x1 + theta[2]*x2 
    y = y_clean + sigma*np.random.randn(len(x1))
    X_lstsq = np.stack((np.ones_like(x1), x1, x2)).T
    param, MSE, rank, singval = np.linalg.lstsq(X_lstsq, y, rcond=None)
    display(theta, param)
    ax.scatter(x1, x2, y, s=10, label='data')
    X1, X2 = np.meshgrid(np.linspace(np.amin(x1), np.amax(x1), num=2), 
                         np.linspace(np.amin(x2), np.amax(x2), num=2))
    ax.plot_surface(X1, X2, param[0] + param[1]*X1 + param[2]*X2, 
                    label='model', alpha=0.25)

interact(update, 
         rseed=IntSlider(continuous_update=False), 
         N=SelectionSlider(options=[10, 100, 1000]),
         sigma=SelectionSlider(options=[0.1, 0.5, 1, 2, 5, 10.]));

### Predicción: Interpolación y Extrapolación

Una vez que el regresor ha sido ajustado se puede usar para hacer predicciónes de la variable dependiente a partir de valores no observados de la variable independiente

- Llamamos interpolación cuando predecimos dentro del rango de nuestros datos
- Llamamos extrapolación cuando predecimos fuera del rango de nuestros datos

In [ ]:
x = np.random.randn(5)
y = 5*x -4 + 2*np.random.randn(len(x))
theta = np.linalg.lstsq(np.stack((np.ones_like(x), x)).T, y, rcond=None)[0]
y_hat = lambda x : np.dot(np.stack((np.ones_like(x), x)).T, theta)
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.scatter(x, y, c='k')
x_plot = np.linspace(np.amin(x), np.amax(x))
ax.plot(x_plot, y_hat(x_plot))
x_plot = np.linspace(np.amax(x), 2*np.amax(x))
ax.plot(x_plot, y_hat(x_plot))
ax.set_xlabel('x')
ax.set_ylabel('y');

### Modelos lineales en sus parámetros pero no en sus entradas

Podemos generalizar la regresión lineal usando funciónes base $\phi_j(\cdot)$ tal que el modelo

$$
y = f_\theta (x) = \sum_{j=0}^N \theta_j \phi_j (x)
$$

#### Regresión lineal con polinomios

Si usamos $\phi_j(x) = x^j$ nos queda

$$
y = f_\theta (x) = \theta_0 + \theta_1 x + \theta_2 x^2 + \ldots
$$

#### Regresión lineal con sinusoides

Si usamos $\phi_j(x) = \cos(2\pi j x)$ nos queda

$$
y = f_\theta (x) = \theta_0 + \theta_1 \cos(2\pi x) + \theta_2 \cos(4 \pi x) + \ldots
$$

In [ ]:
x = np.linspace(0, 2, num=100)
y = 2*np.cos(2.0*np.pi*x) + np.sin(4.0*np.pi*x) + 0.4*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.scatter(x, y)

poly_basis = lambda x,N : np.vstack([x**k for k in range(N)]).T
N = 1
theta = np.linalg.lstsq(poly_basis(x, N), y, rcond=None)[0]
ax.plot(x, np.dot(poly_basis(x, N), theta))

### Sistema infradeterminado

Es aquel sistema que tiene más incognitas (parámetros) que ecuaciones, $N>M$

Este tipo de sistema tiene infinitas soluciones

#### Ejemplo: Dos puntos con polinomio de segundo orden (tres parámetros)

In [ ]:
x = np.array([-2, 2])
y = np.array([4, 4])
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
x_plot = np.linspace(-3, 3, num=100)
thetas = np.zeros(shape=(200, 3))
for i, a in enumerate(np.linspace(-10, 10, num=thetas.shape[0])):
    ax.plot(x_plot, a  + (1 - a/4)*x_plot**2)
    thetas[i:] = [a, 0, (1-a/4)]
ax.scatter(x, y, s=100, c='k', zorder=10);

En este caso $A^T A$ no es invertible 

In [ ]:
A = poly_basis(x, N=3)
display(A)
np.linalg.inv(np.dot(A.T, A))

El problema infradeterminado se resuelve imponiendo una restricción adicional

La más típica es que el vector solución tenga norma mínima

$$
\min_\theta \| x \|_2^2 ~\text{s.a.}~ Ax =b
$$

que se resuelve usando $M$ multiplicadores de Lagrande

$$
\begin{align}
\frac{d}{dx} \| x\|_2^2 + \lambda^T (b - Ax) &= 2x - \lambda^T A  \nonumber \\
&= 2Ax - A A^T \lambda \nonumber \\
&= 2b - A A^T \lambda = 0 \nonumber \\
&\rightarrow \lambda = 2(AA^T)^{-1}b \nonumber \\
&\rightarrow x = \frac{1}{2} A^T \lambda = A^T (A A^T)^{-1} b
\end{align}
$$

donde $A^T (A A^T)^{-1}$ se conoce como la pseudo-inversa "por la derecha"

In [ ]:
x = np.array([-2, 2])
y = np.array([4, 4])
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
x_plot = np.linspace(-3, 3, num=100)
theta = np.dot(np.dot(A.T, np.linalg.inv(np.dot(A, A.T))), y)
ax.plot(x_plot, np.dot(poly_basis(x_plot, N=3), theta))
ax.scatter(x, y, s=100, c='k', zorder=10)
display(theta)
display(thetas[np.argmin(np.sum(thetas**2, axis=1)), :])

In [ ]:
np.linalg.lstsq(A, y, rcond=None)[0]

Investigando nos damos cuenta que `linalg.lstsq` está basado en la función de LAPACK [`dgels`](https://www.math.utah.edu/software/lapack/lapack-d/dgels.html)

dgels usa la pseudo inversa izquierda si $N<M$ o la pseudo inversa derecha si $N>M$

> Se asume que la mejor solución del sistema infradeterminado es la de **mínima norma euclidiana**

## Complejidad, sobreajuste y regularización

Un modelo con más parámetros es más flexible pero también más complejo

Un exceso de flexibilidad no es bueno. Podría ocurrir que:

El modelo se ajuste al ruido y ya no generalice bien ante nuevos datos

> Se dice entonces que el modelo se ha **sobreajustado a los datos**

In [ ]:
x = np.linspace(-3, 3, num=30)
y_clean = np.poly1d([2, -4, -10])(x) # 2*x**2 -4*x -10
y = y_clean + 5*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=( 4, 4), tight_layout=True)
ax.scatter(x, y); 
ax.plot(x, y_clean, lw=2, alpha=.5)
ax.set_xlabel('x'); ax.set_ylabel('y');

A = poly_basis(x, N=29)
theta = np.dot(np.linalg.pinv(A), y)
#theta = np.linalg.lstsq(A, y, rcond=None)[0]
ax.plot(x, np.dot(A, theta), 'k');

In [ ]:
plt.close('all'); fig = plt.figure(figsize=(6, 5), tight_layout=True)
x = np.linspace(-5, 6, num=11); 
x_plot = np.linspace(-5, 6, num=100);
theta = [10, -2, -0.3, 0.1]
X = poly_basis(x, len(theta))
y = np.dot(X, theta)

def update(sigma, rseed, N):
    np.random.seed(rseed); 
    Y = np.dot(X, theta) + sigma*np.random.randn(len(x))
    X2 = poly_basis(x, N)
    theta_hat = np.linalg.lstsq(X2[:10, :], Y[:10], rcond=None)[0]
    print(theta, theta_hat)
    ax = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax.scatter(x[:10], Y[:10], c='r', s=100, label='train data')
    ax.scatter(x[10:], Y[10:], c='g', s=100, label='test data')
    ax.vlines(x[:10], np.dot(X2[:10, :], theta_hat), Y[:10], 'r')  
    ax.vlines(x[10:], np.dot(X2[10:, :], theta_hat), Y[10:], 'g') 
    ax.plot(x, y, 'b--', linewidth=4, label='underlying')
    ax.plot(x_plot, np.dot(poly_basis(x_plot, N), theta_hat), 'k-', linewidth=4, label='model')
    ax.set_ylim([-5, 15]); plt.legend()
    ax = plt.subplot2grid((3, 1), (2, 0))
    ax.plot(x, np.zeros_like(x), 'k--', alpha=0.5)
    ax.scatter(x, Y - np.dot(X2, theta_hat), c='k', s=100); 
    
interact(update, rseed=IntSlider(continuous_update=False), 
         N=SelectionSlider(options=[1, 2, 3, 4, 5, 7, 10]), 
         sigma=SelectionSlider(options=[0.1, 1, 2, 5]));

Tres maneras de evitar el sobreajuste

- Usar modelos de baja complejidad 
- Escoger la complejidad usando validación cruzada
- Usar **regularización**

#### Regularización

Consiste en agregar una penalización adicional al problema 

El ejemplo clásico es pedir que la solución tenga norma mínima

$$
\min_x \|Ax-b\|_2^2 + \lambda \|x\|_2^2
$$

En este caso la solución es

$$
\hat x = (A^T A + \lambda I)^{-1} A^T b
$$

que se conoce como **ridge regression** o **regularización de Tikhonov**

$\lambda$ es un hiper-parámetro del modelo y debe ser escogido por el usuario

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
x = np.linspace(-5, 6, num=50); x_plot = np.linspace(-5, 6, num=200); 
model = np.sin(x)*x + 0.1*x**2
print(repr(theta))

def update(sigma, rseed, lamb, M):
    np.random.seed(rseed); 
    y = model + sigma*np.random.randn(len(x))
    regressor = make_pipeline(PolynomialFeatures(M), Ridge(normalize=True, alpha=lamb))
    regressor.fit(x.reshape(-1, 1), y)
    ax.cla(); ax.plot(x, model, 'b--', linewidth=4, label='underlying')
    ax.plot( x_plot , regressor.predict( x_plot .reshape(-1, 1)), 'k-', linewidth=4, label='model')
    ax.scatter(x, y, c='r', s=30, label='data', zorder=100); plt.legend()

    
interact(update, rseed=IntSlider(continuous_update=False), M=SelectionSlider(options=[1, 2, 3, 5, 10, 20]), 
         sigma=SelectionSlider(options=[0.1, 1, 2, 5]),
         lamb=SelectionSlider(options=[0.0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1., 100000.]));


# Validación

Para escoger la cantidad de parámetros de nuestro modelo o escoger hiper-parámetros como $\lambda$ podemos usar validación cruzada

Antes de ajustar el modelo se particionan los datos en dos conjuntos
1. Conjunto de entrenamiento: Datos que se ocupan para ajustar el modelo
1. Conjuto de validación: Datos que se ocupan para evaluar el modelo

Nos quedamos con el modelo que se desempeña mejor en validación 

Un modelo sobreajustado tiene buen desempeño en entrenamiento y malo en validación

Opciones de más alto nivel para hacer regresión lineal

- `scipy.stats.linregress`
- [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

outliers
https://blog.datarobot.com/ordinary-least-squares-in-python
    https://mmas.github.io/least-squares-fitting-numpy-scipy
        https://docs.scipy.org/doc/scipy-1.3.0/reference/tutorial/linalg.html

https://sbu-python-class.github.io/python-science/